# **Importing the Libraries**


In [6]:
import pandas as pd
import numpy as np
import matplotlib
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns 
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

In [7]:
print(f'matplotlib vesion: {matplotlib.__version__}')
print(f'pandas vesion:     {pd.__version__}')
print(f'numpy vesion:      {np.__version__}')
print(f'seaborn vesion:    {sns.__version__}')
print(f'sklearn vesion:    {skl.__version__}')

matplotlib vesion: 3.2.2
pandas vesion:     1.1.5
numpy vesion:      1.19.5
seaborn vesion:    0.11.1
sklearn vesion:    0.22.2.post1


# **Loading the Data**

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Projects/Silicon Valley 21st Century Education Hackathon/Silicon_Valley_Train.csv')
test = pd.read_csv('/content/drive/MyDrive/Projects/Silicon Valley 21st Century Education Hackathon/Silicon_Vally_Test.csv')
ss = pd.read_csv('/content/drive/MyDrive/Projects/Silicon Valley 21st Century Education Hackathon/SampleSubmission.csv')

In [4]:
train

,ID,countryCode,year,sex,agefrom,ageto,percentage_no_schooling,percentage_primary_schooling,percentage_primary_schooling_completed,percentage_secondary_schooling,percentage_secondary_schooling_completed,percentage_tertiary_schooling,percentage_tertiary_schooling_completed,yr_sch,region_code,literate_pop,target
0,ID_00AMK8K,155,1990,MF,15,999,8.3900,26.270000,15.830000,58.570000,26.420000,6.770000,4.400000,8.540000,Latin America and the Caribbean,7435.0,91.609167
1,ID_00ANS9C,115,2005,MF,25,999,0.6325,6.980833,6.363333,77.683333,49.995833,14.700833,13.546667,11.631667,Europe and Central Asia,14350.0,99.362969
2,ID_00AQU1S,30,2000,M,15,999,68.5300,26.320000,16.020000,4.620000,2.320000,0.540000,0.310000,1.550000,Sub-Saharan Africa,1458.0,31.469890
3,ID_00AVQ7Q,143,1960,MF,15,999,12.5300,69.370000,49.260000,8.130000,1.100000,9.970000,5.510000,7.490000,Latin America and the Caribbean,45.0,86.538462
4,ID_00BAH2C,70,1955,MF,25,999,20.9800,55.900000,22.380000,20.850000,10.240000,2.270000,1.950000,4.890000,Latin America and the Caribbean,2380.0,79.017264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14011,ID_99YTK0T,38,1950,MF,25,999,61.1825,20.794167,13.840833,15.994167,3.747500,2.030000,0.256667,3.115000,Sub-Saharan Africa,4565.0,38.818027
14012,ID_99ZDM2A,77,1990,MF,25,999,4.8800,55.140000,29.530000,30.080000,13.120000,9.900000,5.290000,7.130000,Latin America and the Caribbean,1720.0,95.080155
14013,ID_99ZMQ0L,98,1990,MF,15,999,5.2000,41.400000,21.520000,34.200000,16.560000,18.800000,6.730000,7.100000,East Asia and the Pacific,34077.0,94.398737
14014,ID_99ZNU2T,329,1990,M,25,999,4.5000,47.140000,23.710000,39.450000,27.560000,8.910000,6.220000,8.700000,Europe and Central Asia,2913.0,95.476893
